In [1]:
#Neural network training with syntethic in vivo dopamine and serotonin signals. 
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import keras
import numpy as np
from sklearn.model_selection import train_test_split

print(tf.__version__)

2.3.0


In [2]:
X_filenames = ['synthetic_colorplots'+str(i)+'.npz' for i in range(0,150)]
y_filenames = ['array_of_ratios'+str(i)+'.npz' for i in range (0,150)]


#Shuffling filenames upon importing. 
X_filenames_shuffled, y_filenames_shuffled = shuffle(X_filenames, y_filenames)


# Used this line as our filename array is not a numpy array.
X_filenames_shuffled = np.array(X_filenames_shuffled)
y_filenames_shuffled = np.array(y_filenames_shuffled)
#Divide into train and test. 
X_train_filenames, X_val_filenames, y_train_filenames, y_val_filenames = train_test_split(
    X_filenames_shuffled, y_filenames_shuffled, test_size=0.2, random_state=1)


In [3]:
class my_custom_generator(keras.utils.Sequence):
  
  def __init__(self, filenames, labels):
    self.filenames = filenames
    self.labels = labels
    
    
  def __len__(self):
    return len(self.filenames)
  
  
  def __getitem__(self, idx) :
    batch_x = self.filenames[idx]
    array_x = np.load(batch_x)['a'].reshape(29, 275, 300, 1)
    batch_y = self.labels[idx]
    array_y = np.load(batch_y)['a']
    return array_x, array_y

my_training_batch_generator = my_custom_generator(X_train_filenames, y_train_filenames)
my_validation_batch_generator = my_custom_generator(X_val_filenames, y_val_filenames)

In [4]:
#Define the model. This is only an example and we will need some hyperparametrization. 

def build_and_compile_model():
    model = models.Sequential()
    model.add(layers.Conv2D(275, (3, 3), activation='relu', input_shape=(275, 300, 1)))
    model.add(layers.MaxPooling2D((3, 3)))
    model.add(layers.Conv2D(100, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(1))
    model.summary()
    model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.001), metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model
model = build_and_compile_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 273, 298, 275)     2750      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 91, 99, 275)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 89, 97, 100)       247600    
_________________________________________________________________
flatten (Flatten)            (None, 863300)            0         
_________________________________________________________________
dense (Dense)                (None, 100)               86330100  
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1

In [5]:
model.fit_generator(generator=my_training_batch_generator,
                steps_per_epoch = 3,
                  epochs = 100,
                  verbose = 1,
                 validation_data = my_validation_batch_generator,
                   validation_steps = 5)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
2/3 [===================>..........] - ETA: 1:02 - loss: 29.1420 - root_mean_squared_error: 41.4670

UnknownError:  OSError: [Errno 22] Invalid argument
Traceback (most recent call last):

  File "C:\Users\sergi\anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 244, in __call__
    ret = func(*args)

  File "C:\Users\sergi\anaconda3\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 302, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\sergi\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 827, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\sergi\anaconda3\lib\site-packages\tensorflow\python\keras\engine\data_adapter.py", line 814, in wrapped_generator
    for data in generator_fn():

  File "C:\Users\sergi\anaconda3\lib\site-packages\tensorflow\python\keras\engine\data_adapter.py", line 940, in generator_fn
    yield x[i]

  File "<ipython-input-3-d966183ce053>", line 14, in __getitem__
    array_x = np.load(batch_x)['a'].reshape(29, 275, 300, 1)

  File "C

In [ ]:
model.save('model.h5')

In [ ]:
# TEST WITH LDOPA COLOR PLOTS.
model = tf.keras.models.load_model('model.h5')

In [ ]:
array_x_ld = np.load(r'LD_Colorplots/ld1.npz')['a'].reshape(13, 275, 300, 1)

In [ ]:
plt.plot(model.predict(array_x_ld))

In [ ]:
# It results, again, in an increase of the ratio serotonin to dopamine, which is the oposite to what we are expecting.
#The reason might be the amplitudes of the signals. 